<a href="https://colab.research.google.com/github/ManishKVish/pairs/blob/main/ADF%20CorrelationShip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

**Step 1- Install libraries.**

In [5]:

!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install yfinance
!pip install datetime
!pip install xlsxwriter


import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 882 kB/s 
     |████████████████████████████████| 251 kB 34.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 14.8 MB/s 



**Step 3:- Downloading the data from yahoo finance for ADF testing**

In [14]:

workbook = xlsxwriter.Workbook("C:/Users/manis/Videos/stocks/workbook.xls")
 
# By default worksheet names in the spreadsheet will be
# Sheet1, Sheet2 etc., but we can also specify a name.
worksheet = workbook.add_worksheet("Z-Score")

today1 = dt.datetime.today().strftime('%Y,%m,%d')
# Start from the first cell. Rows and
# columns are zero indexed.
row = 0
col = 0

#start = dt.datetime(2021,6,1).date()
#end = dt.datetime(2021,8,28).date()

#############################################################
#OLD DATE
start = (dt.datetime.today() + relativedelta(months=-11)).date()

#TODAYS DATE
end = dt.datetime.today().date()
#############################################################
list = [
        ["^NSEI","^NSEBANK"],
        ["INFY.NS","TCS.NS"],
        ["TECHM.NS","WIPRO.NS"],
        ["WIPRO.NS","HCLTECH.NS"],
        ["INFY.NS","WIPRO.NS"],
        ["INFY.NS","HCLTECH.NS"],
        ["HDFCBANK.NS","KOTAKBANK.NS"],
 #       [" ICICIBANK.NS","AXISBANK.NS"],
        ["KOTAKBANK.NS","ICICIBANK.NS"],
        ["KOTAKBANK.NS","AXISBANK.NS"],
        ["HDFCBANK.NS","HDFC.NS"],
  #      ["FEDERALBNK.NS"," RBLBANK.NS"],
        ["ACC.NS","AMBUJACEM.NS"],
        ["TATASTEEL.NS","HINDALCO.NS"],
        ["JSWSTEEL.NS","HINDALCO.NS"],
        ["ONGC.NS","GAIL.NS"],
        ["ONGC.NS","NTPC.NS"],
  #      ["BAJFINANCE.NS"," BAJAJFINSV.NS"],
        ["PFC.NS","RECLTD.NS"],
        ["AMARAJABAT.NS","EXIDEIND.NS"], 
        ["APOLLOTYRE.NS","MRF.NS"],
        ["ESCORTS.NS","BALKRISIND.NS"],
        ["DLF.NS","LT.NS"],
        ["DLF.NS","GODREJPROP.NS"],
        ["ULTRACEMCO.NS","GRASIM.NS"],
        ["ULTRACEMCO.NS","SHREECEM.NS"],
        ["tatamotors.NS","BAJAJ-AUTO.NS"],
        ["BAJAJ-AUTO.NS","heromotoco.NS"],
        ["TVSMOTOR.NS","heromotoco.NS"],
        ["DIVISLAB.NS","DRREDDY.NS"],
        ["DIVISLAB.NS","CIPLA.NS"],
        ["HINDPETRO.NS","IOC.NS"],
        ["HINDPETRO.NS","BPCL.NS"],
        ["SBILIFE.NS","HDFCLIFE.NS"],
        ["ASIANPAINT.NS","BERGEPAINT.NS"],
        ["SUNPHARMA.NS","AUROPHARMA.NS"],
        ["ICICIPRULI.NS","HDFCLIFE.NS"],
        ["JUBLFOOD.NS","TATACONSUM.NS"],
        #["ZEEL.NS","SUNTV.NS"],
        #["MSUMI.NS","heromotoco.NS"],
        ]
   
# getting length of list
length = len(list)
   
# Iterating the index
# same as 'for i in range(len(list))'
for i in range(length):
    data=list[i]
    STOCK1=(list[i][0])
    STOCK2=(list[i][1])
    tickers=[
    STOCK1,
    STOCK2
     ]
#==================================================================================
    DF =pd.DataFrame()
    stock = yf.download(data, start, end)
    month =stock['Close'].corr()
#==================================================================================
    for i in tickers:
      df = yf.download(i,start,end,interval = "1d")[['Close']]
      df.columns = [i]                         
      DF = pd.concat([DF,df],axis =1)

    DF = DF.copy().round(5)
    DF


    model = sm.OLS(DF[STOCK1],DF[STOCK2])
    model = model.fit() 
    ' The hedge ratio is %.2f' % model.params[0]


#==================================================================================
    DF['Spread'] = DF.iloc[:,0] - model.params[0] * DF.iloc[:,1]
    DF

#==================================================================================
# To perform ADF Test
    from statsmodels.tsa.stattools import adfuller
# Compute ADF test statistics
    adf = adfuller(DF.Spread, maxlag = 1)
    worksheet.write(row, col, ' ===== '.join(map(str, data)) )
    worksheet.write(row, col + 1, model.params[0])
    worksheet.write(row, col + 2, month.values.tolist()[1][0])
    worksheet.write(row, col + 3, adf[0])
    row += 1
    print("==> "+' ===== '.join(map(str, data)) +"  ######## HEADE RATIO : " + str(model.params[0]) + " ######## Coralation :   "+str(month.values.tolist()[1][0])+"  ########ADF VALUE : "+ str(adf[0]))

#df.to_excel('C:\Users\Ron\Desktop\'Example3.xlsx'', sheet_name='Example3.xlsx' ,index = False)
from xlwt import Workbook
wb = Workbook()
sheet1 = wb.add_sheet("Z-Score")
wb.save("C:/Users/manis/Videos/stocks/workbook.xls")
workbook.close()

KeyboardInterrupt: ignored

99% confidence -3.44

95% Confidence -2.86

90% Confidence -2.56
